In [67]:
import torch
from torch import nn
import torch.nn.functional as F
import random
import os

__all__ = ['Input_conv', 'Net']

class Input_conv(nn.Module):
    def __init__(self, in_channels, out_channels, ks_x=25, ks_y=3):
        super().__init__()
        pad_x = int((ks_x - 1) / 2)
        pad_y = int((ks_y - 1) / 2)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=(ks_y,ks_x),padding=(pad_y, pad_x))
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.pool = nn.MaxPool2d((1, 100))
        self.flat = nn.Flatten(2, -1)

    def forward(self, x1, x2):
        out1 = self.conv1(x1)
        print("Network: ", out1.shape)
        out1 = self.bn1(out1)
        print("Network: ", out1.shape)
        out1 = self.pool(out1)
        print("Network: ", out1.shape)
        out1 = self.flat(out1)
        print("Network: ", out1.shape)
        out1 = torch.unsqueeze(out1, -1)
        print("Network: ", out1.shape)

        out2 = self.conv1(x2)
        out2 = self.bn1(out2)
        out2 = self.pool(out2)
        out2 = self.flat(out2)
        out2 = torch.unsqueeze(out2, -2)
        print("Network: ", out2.shape)

        out = torch.matmul(out1, out2)

        return out

def read_train_data(file_name):
    with open(file_name, 'r', newline='') as f:
        lines = csv.reader(f)
        data = []
        for line in lines:
            data.append(line)

    data = np.array(data).astype(np.float)
    return data

def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import csv

#inputs = torch.rand(1,1,3,1000)
set_seed()
title = ['coop', 'comp', 'singlebystander', 'singleplayer']

for i in range(1):
    inputs = read_train_data('./slapjack_csv/' + title[i] + '/1_1_A.csv')
    inputs = torch.from_numpy(inputs).float()
    inputs = torch.unsqueeze(inputs, 0)
    inputs = torch.unsqueeze(inputs, 0)

    inputs2 = read_train_data('./slapjack_csv/' + title[i] + '/1_1_B.csv')
    inputs2 = torch.from_numpy(inputs2).float()
    inputs2 = torch.unsqueeze(inputs2, 0)
    inputs2 = torch.unsqueeze(inputs2, 0)
    print(inputs.shape)



    model = Input_conv(in_channels=1, out_channels=2)
    output = model(inputs, inputs2)
    print(output.shape)

    output = output.detach().numpy()
    print("max:", np.argmax(output[0,0,:,:], axis=1))
    plt.matshow(output[0,0,:,:], vmin=0, vmax=50)
    plt.colorbar()
    plt.savefig('./' + title[i] + '.png')
    plt.show()

NameError: name 'set_seed' is not defined

In [66]:
out1 = torch.tensor(np.array([[[1], [2], [3]]]))
print("Network: ", out1.shape)
out2 = torch.tensor(np.array([[[4, 5, 6]]]))
print("Network: ", out2.shape)
out = torch.matmul(out1, out2)
print(out)

Network:  torch.Size([1, 3, 1])
Network:  torch.Size([1, 1, 3])
tensor([[[ 4,  5,  6],
         [ 8, 10, 12],
         [12, 15, 18]]], dtype=torch.int32)


In [4]:
ans=0
y= [-12738911,8987884, 27975217,155653128, 267680698]
for i in range(5):
    ans += y[i]*np.power(1.0532, i+1)/np.power(1.013, i+1)
    print(i+1, "year(s): ", ans)

1 year(s):  -13244443.302270485
2 year(s):  -3529052.63424976
3 year(s):  27910595.668972
4 year(s):  209781603.3062585
5 year(s):  534961794.03290856


In [ ]:
  (12738911)   8987884    27975217    155653128    267680698